In [1]:
import pandas as pd 
import datetime
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline


# Предобработка и соединение файлов с номерами и датами

In [2]:
quart = pd.DataFrame()

for file in os.listdir():
    if '$' not in file and 'xlsx' in file and 'АУ.xlsx' not in file:
        dict_df = pd.read_excel(file, sheet_name=None, dtype=str)
        for key in dict_df.keys():
            name = (file[: file.find('.')] + ' ' + key).strip()
            temp_df = dict_df[key][['Дата', 'Клиент: телефон']]
            temp_df['Источник'] = [name] * temp_df.shape[0]
            quart = quart.append(temp_df)

C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [3]:
quart = quart.reset_index(drop=True)


In [4]:
pattern = re.compile(r'([^ ]+)( [^ ]+)')


def string(s):
    pattern = re.compile(r'([^ ]+)( [^ ]+)')
    if pattern.findall(s):
        return pattern.findall(s)[0][0]
    else:
        return s
    
quart['Дата_new'] = quart['Дата'].apply(string)


In [5]:
def to_date(x):
    try:
        x = datetime.datetime.strptime(x, '%Y-%m-%d').date()
    except:
        x = datetime.datetime.strptime(x, '%d.%m.%Y').date()
    return x    

In [6]:
quart['Дата_new'] = quart['Дата_new'].apply(to_date)

In [7]:
quart.drop('Дата', axis=1, inplace=True)

In [8]:
quart = quart.rename(columns={'Дата_new': 'Дата'})

In [9]:
quart

,Клиент: телефон,Источник,Дата
0,79134782306,Звонки 4кв Докдок НСК Sheet1,2020-12-31
1,74952367276,Звонки 4кв Докдок НСК Sheet1,2020-12-31
2,74952367276,Звонки 4кв Докдок НСК Sheet1,2020-12-31
3,79134782306,Звонки 4кв Докдок НСК Sheet1,2020-12-31
4,79134782306,Звонки 4кв Докдок НСК Sheet1,2020-12-31
...,...,...,...
574,74952367276,Звонки 4кв Докдок НСК Sheet1,2020-10-11
575,74952367276,Звонки 4кв Докдок НСК Sheet1,2020-10-11
576,74952367276,Звонки 4кв Докдок НСК Sheet1,2020-10-10
577,74952367276,Звонки 4кв Докдок НСК Sheet1,2020-10-10


In [10]:
quart = quart.dropna().reset_index(drop=True)


# re - телефоны

In [11]:
phones = quart['Клиент: телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()

for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y    

In [12]:
quart['длина'] = quart['Клиент: телефон'].apply(len)

In [13]:
quart['Чистые телефоны'] = phones2

In [14]:
quart['Чистые телефоны'] = quart['Чистые телефоны'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)

In [15]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [16]:
quart[quart['длина чистые'] != 11]






,Клиент: телефон,Источник,Дата,длина,Чистые телефоны,длина чистые


In [17]:
quart = quart.drop(quart[quart['длина чистые'] != 11].index)

In [18]:
def clean_number(row):
    if row['длина чистые'] == 14:
        row['Чистые телефоны'] = '7' + row['Чистые телефоны'][4:]
    return row
        

In [19]:
quart = quart.apply(clean_number, axis=1)

In [20]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [21]:
pattern_7 = r'^7\d{10}$'
quart['соответствие'] = quart['Чистые телефоны'].apply(lambda x: bool(re.match(pattern_7, x)))

In [22]:
quart[~quart['соответствие']]

,Клиент: телефон,Источник,Дата,длина,Чистые телефоны,длина чистые,соответствие


In [83]:
quart.drop([40], inplace=True)


In [23]:
quart.drop(['Клиент: телефон', 'длина', 'длина чистые', 'соответствие'], axis=1, inplace=True)

In [25]:
quart = quart.sort_values(by='Дата', ignore_index=True)
quart = quart.drop_duplicates(subset='Чистые телефоны', ignore_index=True)
quart

,Источник,Дата,Чистые телефоны
0,Звонки 4кв Докдок НСК Sheet1,2020-10-08,78123804446
1,Звонки 4кв Докдок НСК Sheet1,2020-10-10,74952367276
2,Звонки 4кв Докдок НСК Sheet1,2020-10-19,79134875808
3,Звонки 4кв Докдок НСК Sheet1,2020-10-19,79831257696
4,Звонки 4кв Докдок НСК Sheet1,2020-10-21,73833832731
5,Звонки 4кв Докдок НСК Sheet1,2020-10-22,79237146595
6,Звонки 4кв Докдок НСК Sheet1,2020-10-27,73833500846
7,Звонки 4кв Докдок НСК Sheet1,2020-10-27,79538579444
8,Звонки 4кв Докдок НСК Sheet1,2020-10-28,79137929197
9,Звонки 4кв Докдок НСК Sheet1,2020-11-02,79581113944


# АУ

In [26]:
au_telefons = pd.read_excel('АУ.xlsx')
au_telefons = au_telefons.fillna('')

In [27]:
dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

In [28]:
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if len([x for x in j if x.isdigit()]) == 10:
            key[i] = '7' + ''.join([x for x in j if x.isdigit()])    
        if len([x for x in j if x.isdigit()]) == 11:
            temp_arr = ''.join([x for x in j if x.isdigit()])
            if temp_arr[0] == '8':
                key[i] = '7' + temp_arr[1:]
            elif temp_arr[0] == '7':    
                key[i] = temp_arr




for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  

In [29]:
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j

In [30]:
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)        

In [31]:
lst_search

['79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 ' 79139133489 ',
 '73832066471 79232568200 ',
 '73832066471 79232568200 ',
 '73832066471 79232568200 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79538753603 ',
 ' 79232403980 ',
 ' 79139396377 ',
 ' 79139396377 ',
 '73832166557 79139262716 

# Сравнение

In [32]:
lst_with_coincidenc = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc.append('')
    for i in quart['Чистые телефоны']:
        if str(i) in row:
            lst_with_coincidenc[index] = str(i)
            break
            
            
    

In [33]:
au_telefons['Совпадение'] = lst_with_coincidenc

In [34]:
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']

In [35]:
au_telefons_final = au_telefons_final.merge(quart, left_on='Совпадение', right_on='Чистые телефоны', how='left')

In [36]:
au_telefons_final = au_telefons_final.fillna('')

In [37]:
au_telefons_final['Старый/Новый'] = au_telefons_final['Дата приема'] == au_telefons_final['Минимальная дата приема Пациента (без удаленных)']
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})
new_clients_lst = au_telefons_final[au_telefons_final['Старый/Новый'] == 'Новый']['Номер карты'].unique()
au_telefons_final.drop('Старый/Новый', axis=1, inplace=True)
au_telefons_final['Старый/Новый'] = au_telefons_final['Номер карты'].isin(new_clients_lst)
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})


In [38]:
au_telefons_final

,Номер карты,Пациент,Дата приема,Минимальная дата приема Пациента (без удаленных),Наименование услуги,Отделение из профиля Исполнителя,Телефон домашний,Телефон мобильный,Телефон рабочий,Категория клиента,Sum([Стоимость полная]),Sum([Количество]),Филиал,Совпадение,Источник,Дата,Чистые телефоны,Старый/Новый
0,540053166,Тайлакова Зухра Рустамовна,2020-11-13,2020-01-20,Консультация врача - эндокринолога первичная в...,Терапевтическое отделение,,+7(952)908-53-66,,НАЛ,1400.0,1,Некрасова,79529085366,Звонки 4кв Докдок НСК Sheet1,2020-11-13,79529085366,Старый
1,540053166,Тайлакова Зухра Рустамовна,2020-11-13,2020-01-20,УЗИ щитовидной железы и лимфоузлов с ЦДК в амб...,Отделение ультразвуковой и функциональной диаг...,,+7(952)908-53-66,,НАЛ,900.0,1,Некрасова,79529085366,Звонки 4кв Докдок НСК Sheet1,2020-11-13,79529085366,Старый
2,540060992,Притчин Егор Евгеньевич,2020-10-19,2020-10-19,Консультация врача - дерматолога первичная в ...,Дерматологическое отделение,,+7(913)487-58-08,,НАЛ,1400.0,1,Некрасова,79134875808,Звонки 4кв Докдок НСК Sheet1,2020-10-19,79134875808,Новый
3,540061789,Кузьмин Константин Николаевич,2020-10-28,2020-10-28,Консультация врача - дерматолога первичная в ...,Дерматологическое отделение,,+7(913)792-91-97,,НАЛ,1400.0,1,Некрасова,79137929197,Звонки 4кв Докдок НСК Sheet1,2020-10-28,79137929197,Новый
4,540061789,Кузьмин Константин Николаевич,2020-10-28,2020-10-28,Внутримышечная инъекция в амбулатории (с учето...,Сестринская служба: Отделение процедурных медс...,,+7(913)792-91-97,,НАЛ,250.0,1,Некрасова,79137929197,Звонки 4кв Докдок НСК Sheet1,2020-10-28,79137929197,Новый
5,540062362,Кузнецов Артем Евгеньевич,2020-12-02,2020-12-02,Консультация врача - дерматолога первичная в ...,Дерматологическое отделение,,+7(983)600-46-00,,НАЛ,1400.0,1,Некрасова,79836004600,Звонки 4кв Докдок НСК Sheet1,2020-11-30,79836004600,Новый
6,540062661,Неволина Любовь Ивановна,2020-11-11,2020-11-11,Консультация врача - невролога первичная на дому,Взрослое неврологическое отделение,,+7(923)122-10-14,,НАЛ,2700.0,1,Некрасова,79231221014,Звонки 4кв Докдок НСК Sheet1,2020-11-11,79231221014,Новый
7,540062949,Поляков Валерий Иванович,2020-11-16,2020-11-16,Консультация врача - уролога первичная на дому,Урологическое отделение,,+7(913)948-87-63,,НАЛ,2700.0,1,Некрасова,79139488763,Звонки 4кв Докдок НСК Sheet1,2020-11-18,79139488763,Новый
8,540062949,Поляков Валерий Иванович,2020-12-01,2020-11-16,Консультация врача - уролога повторная на дому,Урологическое отделение,,+7(913)948-87-63,,НАЛ,2200.0,1,Некрасова,79139488763,Звонки 4кв Докдок НСК Sheet1,2020-11-18,79139488763,Новый
9,540062949,Поляков Валерий Иванович,2020-12-01,2020-11-16,Удаление катетера Фолея на дому,Урологическое отделение,,+7(913)948-87-63,,НАЛ,1200.0,1,Некрасова,79139488763,Звонки 4кв Докдок НСК Sheet1,2020-11-18,79139488763,Новый


In [39]:
au_telefons_final = au_telefons_final[au_telefons_final['Дата'] <= au_telefons_final['Дата приема']]

In [40]:
au_telefons_final['Уникальный клиент'] = (~au_telefons_final.duplicated(subset=['Номер карты'], keep='first')).astype(int)


C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
au_telefons_final.to_excel('Обработано-НСК_докдок.xlsx', index=False)